In [3]:
!pip install rasterio
import rasterio
from rasterio.windows import Window
import numpy as np
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 11.0 MB/s eta 0:00:00m eta 0:00:010:01:01


/home/r/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# 1. Split water masks in 256x256 thumbnails and filter them according to water presence

I discard thumbnails with only land or only water. 

I chose (arbitrary) to keep only thumbnails with at least 10% of water and at least 10% of land.  

## 1.1. Plot masks in jpg for quick visual assessment of their quality

In [ ]:
def split_selected_masks_for_assessment(image_path, output_folder, size=256): 
    selected_indexes=[]
    
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        for i in range(0, w, size): 
            for j in range(0, h, size): 
                window = Window(i, j, min(size, w - i), min(size, h - j))
                thumbnail = src.read(window = window)
                
                # keep only thumbnails with at least 10% of water and at least 10% of land 
                nb_water_pixels = sum(sum(sum(thumbnail)))
                
                if (nb_water_pixels > 65536 * (10/100)) and (nb_water_pixels < 65536 * (90/100)):  
                    output_path = f"{output_folder}/tile_{i}_{j}.png"
                    selected_indexes.append((i,j))
                    with rasterio.open(
                        output_path, 
                        'w', 
                        driver="PNG", 
                        height = 256, 
                        width = 256, 
                        count = src.count, 
                        dtype = src.dtypes[0]
                    ) as dst: 
                        dst.write(thumbnail*255)
                        
    return selected_indexes

In [ ]:
image_path = "/home/llalla/Documents/SWOT/dataset/S1S2Water/5/sentinel12_s2_5_msk.tif"
output_folder = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/for_assessment/"

In [ ]:
selected_indexes = split_selected_masks_for_assessment(image_path, output_folder)
print(len(selected_indexes))
print(selected_indexes)

### TO DO : superimpose mask and RGB s2 

## 1.2. Save masks as tif for training data

In [ ]:
def split_selected_masks(image_path, prefix, output_folder, size=256): 
    selected_indexes=[]
    
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        for i in range(0, w, size): 
            for j in range(0, h, size): 
                window = Window(i, j, min(size, w - i), min(size, h - j))
                thumbnail = src.read(window = window)
                
                # keep only thumbnails with at least 10% of water and at least 10% of land 
                nb_water_pixels = sum(sum(sum(thumbnail)))
                
                if (nb_water_pixels > 65536 * (10/100)) and (nb_water_pixels < 65536 * (90/100)):  
                    output_path = f"{output_folder}/{prefix}_mask_tile_{i}_{j}.tif"
                    selected_indexes.append((i,j))
                    with rasterio.open(
                        output_path, 
                        'w', 
                        driver="GTiff", 
                        height = 256, 
                        width = 256, 
                        count = src.count, 
                        dtype = src.dtypes[0]
                    ) as dst: 
                        dst.write(thumbnail)
                        
    return selected_indexes

In [ ]:
image_path = "/home/llalla/Documents/SWOT/dataset/S1S2Water/13/sentinel12_s2_13_msk.tif"
output_folder = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/13_masks/"

In [ ]:
selected_indexes = split_selected_masks(image_path, '13', output_folder)
print(len(selected_indexes))
print(selected_indexes)

### indexes for selected masks : 

In [ ]:
selected_indexes[:9]

[(0, 512),
 (0, 3584),
 (0, 7680),
 (0, 8960),
 (0, 9216),
 (0, 9472),
 (0, 9728),
 (0, 9984),
 (0, 10240),
 (0, 10496),
 (0, 10752),
 (256, 6912),
 (256, 7936),
 (256, 9728),
 (256, 9984),
 (256, 10240),
 (256, 10496),
 (256, 10752),
 (512, 9984),
 (512, 10240),
 (512, 10496),
 (768, 1280),
 (768, 1536),
 (768, 7936),
 (768, 10752),
 (1536, 6912),
 (1536, 8192),
 (1536, 8448),
 (1792, 6656),
 (1792, 6912),
 (2048, 6656),
 (2048, 6912),
 (2304, 6912),
 (2304, 7168),
 (2560, 5376),
 (2560, 6400),
 (2560, 6656),
 (2560, 6912),
 (2560, 7168),
 (2816, 5120),
 (2816, 5888),
 (2816, 6144),
 (2816, 6400),
 (2816, 6656),
 (2816, 6912),
 (2816, 7168),
 (3072, 5376),
 (3072, 5632),
 (3072, 5888),
 (3072, 6144),
 (3072, 7168),
 (3072, 10752),
 (3328, 5120),
 (3328, 5376),
 (3328, 6912),
 (3328, 7168),
 (3584, 0),
 (3584, 5120),
 (3584, 5376),
 (3584, 6656),
 (3584, 6912),
 (3840, 5376),
 (3840, 6144),
 (3840, 6400),
 (4096, 5376),
 (4096, 5632),
 (4096, 6144),
 (4352, 5632),
 (4352, 5888),
 (4352, 6144),
 (4608, 5888),
 (4608, 6144),
 (4608, 6400),
 (4608, 6656),
 (4864, 6144),
 (4864, 6656),
 (4864, 6912),
 (6656, 5632),
 (6656, 6144),
 (6912, 4352),
 (6912, 4608),
 (6912, 5632),
 (6912, 5888),
 (7168, 7168),
 (7424, 5376),
 (7680, 1792),
 (7936, 1792),
 (8192, 6400),
 (8192, 6656),
 (8448, 5888),
 (8448, 6656),
 (8448, 6912),
 (8704, 6144),
 (8960, 10240),
 (8960, 10496),
 (9216, 1536),
 (9216, 10496),
 (10752, 768),
 (10752, 1280),
 (10752, 5376)]

# 2. Crop s2 image accordingly

Sentinel-2 image bands
{"Blue": 0, "Green": 1, "Red": 2, "NIR": 3, "SWIR1": 4, "SWIR2": 5}

S2 Unit: TOA reflectance (scaled by factor 10000)

(TOA = Top Of Atmosphere)

https://github.com/MWieland/s1s2_water/tree/main 

Uncomment the following if you want to see what each part does : 

In [ ]:
# image_path = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/16/sentinel12_s2_16_img.tif"
# with rasterio.open(image_path) as src: 
#     w, h = src.width, src.height
#     size = 256
#     i=0
#     j=3584
#     window = Window(i, j, min(size, w - i), min(size, h - j))
#     thumbnail = src.read(window = window)

# print(thumbnail.shape) # 6 x 256 x 256 
# # {"Blue": 0, "Green": 1, "Red": 2, "NIR": 3, "SWIR1": 4, "SWIR2": 5}
# print(thumbnail)

In [ ]:
# bands = []
# for i in range(thumbnail.shape[0]):
#     bands.append(thumbnail[i, :, :].astype(np.float32) / 10000)
    
# scaled_thumbnail = np.dstack(bands)
# print(scaled_thumbnail.shape)
# new_scaled_thumbnail= np.moveaxis(scaled_thumbnail, -1, 0)
# print(new_scaled_thumbnail.shape)
# print(new_scaled_thumbnail[:3, :, :].shape)
# new_scaled_thumbnail[:3, :, :].max()

In [ ]:
# bands = []
# for i in range(thumbnail.shape[0]):
#     x = (thumbnail[i, :, :].astype(np.float32) / 10000)
#     bands.append((x - x.min()) / (x.max() - x.min())) 
    
# normalized_thumbnail = np.dstack(bands)

# print(normalized_thumbnail.shape)
# print(normalized_thumbnail.max())
# normalized_thumbnail

# new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)
# print(new_normalized_thumbnail.shape)
# print(new_normalized_thumbnail[:3, :, :].shape)
# new_normalized_thumbnail[:3, :, :].max()

In [ ]:
def scale_min_max(array, min=0, max=10000):
    bands = []
    for i in range(array.shape[2]):
        bands.append(array[:, :, i].astype(np.float32) / max)
    return np.dstack(bands)

In [ ]:
def split_selected_s2_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 10000
                x = (thumbnail[k, :, :].astype(np.float32) / 10000)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s2_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 6, # 6 bands
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

In [ ]:
image_path = "/home/llalla/Documents/SWOT/dataset/S1S2Water/13/sentinel12_s2_13_img.tif"
output_folder = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/13_s2/"

In [ ]:
split_selected_s2_images(image_path, '13', selected_indexes, output_folder)

### TO DO : normalisation of thumbnailss = divide by std ? check distribution

# 3. Crop s1 image accordingly

Sentinel-1 image bands {"VV": 0, "VH": 1}

S1 Unit: dB (scaled by factor 100)

https://github.com/MWieland/s1s2_water/tree/main 

Uncomment the following if you want to see what each part does : 

In [ ]:
# image_path = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/11/sentinel12_s1_11_img.tif"
# with rasterio.open(image_path) as src: 
#     w, h = src.width, src.height
#     size = 256
#     i=0
#     j=0
#     window = Window(i, j, min(size, w - i), min(size, h - j))
#     thumbnail = src.read(window = window)

# print(thumbnail.shape) # 2 x 256 x 256 : {"VV": 0, "VH": 1}
# print(thumbnail)

In [ ]:
# s1_0 = thumbnail[0, : , :] # 1 band image
# s1_1 = thumbnail[1, : , :] # 1 band image

# fig, ax = plt.subplots(2, 1, figsize=(20, 6))
# fig.tight_layout()
# ax[0].imshow(s1_0)
# ax[0].set_title('s1_0 Images')    
# ax[1].imshow(s1_1)
# ax[1].set_title('s1_1 Images')

In [ ]:
# bands = []
# for k in range(thumbnail.shape[0]):
#     # re scale all bands : fisrt divide by 100
#     x = (thumbnail[k, :, :].astype(np.float32) / 100)
#     # normalise between 0 and 1
#     bands.append((x - x.min()) / (x.max() - x.min())) 
# normalized_thumbnail = np.dstack(bands)

In [ ]:
# new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

In [ ]:
def split_selected_s1_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 100
                x = (thumbnail[k, :, :].astype(np.float32) / 100)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s1_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 2, 
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

In [ ]:
image_path = "/home/llalla/Documents/SWOT/dataset/S1S2Water/5/sentinel12_s1_5_img.tif"
output_folder = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/5_s1/"

split_selected_s1_images(image_path, '5', selected_indexes, output_folder)



# 4. All fonctions combined to split all data at once

### split masks = keep only thumbnails with at least 10% of water and at least 10% of land 

In [4]:
def split_selected_masks(image_path, prefix, output_folder, size=256): 
    selected_indexes=[]
    
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        for i in range(0, w, size): 
            for j in range(0, h, size): 
                window = Window(i, j, min(size, w - i), min(size, h - j))
                thumbnail = src.read(window = window)
                
                # keep only thumbnails with at least 10% of water and at least 10% of land 
                nb_water_pixels = sum(sum(sum(thumbnail)))
                
                if (nb_water_pixels > 65536 * (10/100)) and (nb_water_pixels < 65536 * (90/100)):  
                    output_path = f"{output_folder}/{prefix}_mask_tile_{i}_{j}.tif"
                    selected_indexes.append((i,j))
                    with rasterio.open(
                        output_path, 
                        'w', 
                        driver="GTiff", 
                        height = 256, 
                        width = 256, 
                        count = src.count, 
                        dtype = src.dtypes[0]
                    ) as dst: 
                        dst.write(thumbnail)
                        
    return selected_indexes

### split matching s2, keep 6bands

In [5]:
def scale_min_max(array, min=0, max=10000):
    bands = []
    for i in range(array.shape[2]):
        bands.append(array[:, :, i].astype(np.float32) / max)
    return np.dstack(bands)

def split_selected_s2_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 10000
                x = (thumbnail[k, :, :].astype(np.float32) / 10000)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s2_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 6, # 6 bands
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

### split s1

In [6]:
def split_selected_s1_images(image_path, prefix, selected_indexes, output_folder, size=256): 
    # open "big" image
    with rasterio.open(image_path) as src: 
        w, h = src.width, src.height
        
        # crop thumbnail
        for (i,j) in selected_indexes: 
            window = Window(i, j, min(size, w - i), min(size, h - j))
            thumbnail = src.read(window = window)
 
            bands = []
            for k in range(thumbnail.shape[0]):
                # re scale all bands : fisrt divide by 100
                x = (thumbnail[k, :, :].astype(np.float32) / 100)
                # normalise between 0 and 1
                bands.append((x - x.min()) / (x.max() - x.min())) 
                
            normalized_thumbnail = np.dstack(bands)
            new_normalized_thumbnail= np.moveaxis(normalized_thumbnail, -1, 0)

            output_path = f"{output_folder}/{prefix}_s1_tile_{i}_{j}.tif"

            with rasterio.open(
                output_path, 
                'w', 
                driver="GTiff", 
                height = 256, 
                width = 256, 
                count = 2, 
                dtype = np.float32
            ) as dst: 
                dst.write(new_normalized_thumbnail)

In [7]:
def split_data(input_folder, output_folder, prefix): 
    
    mask_path = input_folder + prefix + "/sentinel12_s2_"+ prefix + "_msk.tif"
    output_folder_mask = output_folder + "masks/"
    selected_indexes = split_selected_masks(mask_path, prefix, output_folder_mask)
    
    s2_path = input_folder + prefix + "/sentinel12_s2_"+ prefix + "_img.tif"
    output_folder_s2 = output_folder + "s2/"
    split_selected_s2_images(s2_path, prefix, selected_indexes, output_folder_s2)
    
    s1_path = input_folder + prefix + "/sentinel12_s1_"+ prefix + "_img.tif"
    output_folder_s1 = output_folder + "s1/"
    split_selected_s1_images(s1_path, prefix, selected_indexes, output_folder_s1)
    
    return len(selected_indexes)

In [8]:
prefix = '13'
# output_folder = "/home/llalla/Documents/SWOT/tuto_unet/data_folder/"
# input_folder = "/home/llalla/Documents/SWOT/dataset/S1S2Water/"
output_folder = '/home/r/pao_ciel_mon_point_deau/data_folder'
input_folder = '/home/r/pao_ciel_mon_point_deau/S1S2Water'

split_data(input_folder, output_folder, prefix)

RasterioIOError: /home/r/pao_ciel_mon_point_deau/S1S2Water13/sentinel12_s2_13_msk.tif: No such file or directory